# Unstructured grid

## Reading data from a mesh.

Here we read the data from the ocean model MITgcm LLC4320

In [ ]:
import intake

cat_url = ("https://raw.githubusercontent.com/pangeo-data/"
           "pangeo-datastore/master/intake-catalogs/ocean/llc4320.yaml")
cat = intake.Catalog(cat_url)

Reading of the grid positions. The grid is under-sampled because the volume of data is very large.

In [ ]:
grid = cat.LLC4320_grid.to_dask()
grid

In [ ]:
subsampling = slice(0, None, 8)

lons = grid["XC"].isel(i=subsampling, j=subsampling).values
lats = grid["YC"].isel(i=subsampling, j=subsampling).values

In [ ]:
del grid

Reading data. We get one time frame only.

In [ ]:
ssh = cat.LLC4320_SSH.to_dask()
ssh = ssh["Eta"].isel(time=0, i=subsampling, j=subsampling).values

## R\*Tree

The interpolation of this object is based on an [R\*Tree](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree) structure. To begin with, we start by building this object. By default, this object considers WGS-84 geodetic coordinate system. But you can define another one using class [System](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.geodetic.html#pyinterp.geodetic.System).

In [ ]:
import pyinterp
mesh = pyinterp.RTree()

## Creating the search tree

Then, we will insert points into the tree. The class allows you to insert points using two algorithms. The first one called [packing](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.packing) allows you to insert the values in the tree at once. This mechanism is the recommended solution to create an optimized in-memory structure, both in terms of construction time and queries. When this is not possible, you can insert new information into the tree as you go along using the [insert](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.insert) method.

In [ ]:
import numpy as  np

mesh.packing(
    np.vstack((lons.flatten(), lats.flatten())).T,
    ssh.flatten())

When the tree is created, you can interpolate data with two algorithms:
* [Inverse Distance Weighting](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.inverse_distance_weighting) or IDW
* [Radial Basis Function](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.radial_basis_function) or RBF

Yon can also search the [nearest neighbors](https://pangeo-pyinterp.readthedocs.io/en/latest/api/pyinterp.rtree.html#pyinterp.rtree.RTree.query) on the tree.

In this example, we will under-sample the source grid at 1/32 degree over an area of the globe.

In [ ]:
x0, x1 = 80, 170
y0, y1 = -45, 30
res = 1 / 32.0
mx, my = np.meshgrid(
    np.arange(x0, x1, res),
    np.arange(y0, y1, res),
    indexing="ij")

idw_eta, neighbors = mesh.inverse_distance_weighting(
    np.vstack((mx.flatten(), my.flatten())).T,
    within=False,    # Extrapolation is forbidden
    radius=55000,    # In a radius of 5.5 Km
    k=8,             # We are looking for at most 8 neighbours
    num_threads=0)
idw_eta = idw_eta.reshape(mx.shape)

rbf_eta, neighbors = mesh.radial_basis_function(
    np.vstack((mx.flatten(), my.flatten())).T,
    within=False,    # Extrapolation is forbidden
    k=11,            # We are looking for at most 8 neighbours
    num_threads=0)
rbf_eta = rbf_eta.reshape(mx.shape)

Let's visualize our interpolated data

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.mpl.ticker as cticker
%matplotlib inline

fig = plt.figure(figsize=(18, 9))
lon_formatter = cticker.LongitudeFormatter(zero_direction_label=True)
lat_formatter = cticker.LatitudeFormatter()
ax = fig.add_subplot(121, projection=ccrs.PlateCarree())
ax.pcolormesh(mx, my, idw_eta, cmap='terrain',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_xticks(np.arange(x0, x1, 10.0))
ax.set_yticks(np.arange(y0, y1, 10))
ax.set_title("Eta (IDW)")

ax = fig.add_subplot(122, projection=ccrs.PlateCarree())
ax.pcolormesh(mx, my, rbf_eta, cmap='terrain',
              transform=ccrs.PlateCarree())
ax.coastlines()
ax.xaxis.set_major_formatter(lon_formatter)
ax.yaxis.set_major_formatter(lat_formatter)
ax.set_xticks(np.arange(x0, x1, 10.0))
ax.set_yticks(np.arange(y0, y1, 10))
ax.set_title("Eta (RBF)")